In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import random
import torch, torch.nn as nn
from PIL import Image
from tqdm import tqdm
from torch.optim import AdamW
from torch.utils import data
from torchvision import transforms, models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.functional as F
from torchmetrics.detection import MeanAveragePrecision
import pickle

In [2]:
DISABLE_TQDM = True
torch.manual_seed(42)

In [3]:
def train_test_split(path):
    train_path = path + "train_images/"
    train_list = [os.path.join(train_path, img) for img in os.listdir(train_path)]
    random.shuffle(train_list)
    threshold = int(0.8 * len(train_list))
    valid_list = train_list[threshold:]
    train_list = train_list[:threshold]
    test_path = path + "test_images/"
    test_list = [os.path.join(test_path, img) for img in os.listdir(test_path)]
    return train_list, valid_list, test_list

In [15]:
def compute_annotation_map(annotations):
    annotation_map = {}
    for _, row in annotations.iterrows():
        boxes = annotation_map.get(row['image_id'], [])
        boxes.append((row['lesion_type'], [row['xmin'] if row['xmin']>=0 else -1,
                                           row['ymin'] if row['ymin']>=0 else -1,\
                                           row['xmax'] if row['xmax']>=0 else -1,\
                                           row['ymax'] if row['ymax']>=0 else -1\
                                           ]))
        annotation_map[row['image_id']] = boxes
    return annotation_map

In [5]:
def split_annotation_map(annotation_map, train, valid):
    
    train_ids = [image.split('/')[-1].split('.')[0] for image in train]
    valid_ids = [image.split('/')[-1].split('.')[0] for image in valid]

    train_dict = {
        image: annotation_map[image] for image in train_ids
    }
    valid_dict = {
        image: annotation_map[image] for image in valid_ids
    }
    return train_dict, valid_dict

In [6]:
class SpineObjectDetection(data.Dataset):
    
    def __init__(self, root_dir, annotation_map, anomaly_map, image_id_map, transform):
        self.img_paths = root_dir
        self.img_paths.sort()
        self.image_ids = [image.split('/')[-1].split('.')[0] for image in self.img_paths]
        
        self.annotation_map = annotation_map
        self.anomaly_map = anomaly_map
        self.image_id_map = image_id_map
        self.transform = transform       
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx])
        image_id = self.image_ids[idx]
        
        image_width = image.size[0]
        image_height = image.size[1]
        
        image = transforms.Resize(np.random.randint(640,800))(image)
        transformed_w = image.size[0]
        transformed_h = image.size[1]

        labels, boxes, area = [], [], []
        for label, box in self.annotation_map[image_id]:
            labels.append(self.anomaly_map[label])
            
            if box[0] == -1 and box[1] == -1:
                boxes.append([0,0,transformed_w,transformed_h])
            else:
                boxes.append([
                    (box[0]/image_width)  * transformed_w,
                    (box[1]/image_height) * transformed_h,
                    (box[2]/image_width)  * transformed_w,
                    (box[3]/image_height) * transformed_h
                ])

            area.append((boxes[-1][2] - boxes[-1][0]) * (boxes[-1][3] - boxes[-1][1]))

        labels = torch.as_tensor(labels, dtype=torch.int64)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)       
        
        image, boxes = self.transform(image, boxes)
        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": torch.tensor(self.image_id_map[image_id])
        }
        
        return image, target

In [7]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, boxes):
        for t in self.transforms:
            img, boxes = t(img,boxes)
        return img, boxes

    def __repr__(self) -> str:
        format_string = self.__class__.__name__ + "("
        for t in self.transforms:
            format_string += "\n"
            format_string += f"    {t}"
        format_string += "\n)"
        return format_string

class RandomHorizontalFlip():
    
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def __call__(self, img, boxes):
        if torch.rand(1) < self.p:
            
            img_width = img.size[0]
            
            boxes[:,0] = img_width - boxes[:,0]
            boxes[:,2] = img_width - boxes[:,2]
            boxes_w = torch.abs(boxes[:,0] - boxes[:,2])
             
            boxes[:,0] -= boxes_w
            boxes[:,2] += boxes_w

            return F.hflip(img),boxes
        return img,boxes

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(p={self.p})"
    
class ToTensor:
    def __call__(self, img, boxes):
        return F.to_tensor(img), boxes

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}()"

In [8]:
def collate_fn(batch):
    return tuple(zip(*batch))

def create_faster_rcnn_model(num_classes, trainable_backbone_layers=3):
    model = models.detection.fasterrcnn_resnet50_fpn(
        pretrained=True,
        trainable_backbone_layers=trainable_backbone_layers,
        min_size=640,
        max_size=2699
    )
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
    return model

def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    _, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        
def show_tranformed_image(loader, number):
    images, targets = next(iter(loader))
    #images = [image for image in images]
    targets = [{k: v for k, v in t.items()} for t in targets]
    for image, target in list(zip(images,targets))[:number]:
        result = draw_bounding_boxes(
            (image*255).byte(),
            target['boxes'],
            width=5,
            colors=['white']*len(target['boxes'])
        )
        show(result)

def save_object(path,file_name,obj):
    with open(os.path.join(path,file_name + ".pickle"), 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)

def load_object(path,file_name):
    with open(os.path.join(path,file_name + ".pickle"), 'rb') as file:
        return pickle.load(file)

In [9]:
class SaveBestModel:
    def __init__(self, model_name, path="/"):
        self.best_valid_loss = float('inf')
        self.path = path
        self.model_name = model_name
        
    def update(self, model, current_valid_loss):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            torch.save(model, os.path.join(self.path,self.model_name))
            print(f"Saved Model. Best validation loss: {self.best_valid_loss}")
            
    def fetch(self):
        return torch.load(
            os.path.join(self.path, self.model_name)
        )
    
class LossHistory:
    def __init__(self,path,file_name):
        self.loss = {
            'total_loss': [],
            'classifier_loss' : [],
            'box_reg_loss':[],
            'objectness_loss': [],
            'rpn_box_reg_loss':[]
        }
        self.file_name = file_name
        self.path = path
    
    def update(self,total_loss,classifier_loss,box_reg_loss,objectness_loss,rpn_box_reg_loss):
        self.loss['total_loss'].append(total_loss)
        self.loss['classifier_loss'].append(classifier_loss)
        self.loss['box_reg_loss'].append(box_reg_loss)
        self.loss['objectness_loss'].append(objectness_loss)
        self.loss['rpn_box_reg_loss'].append(rpn_box_reg_loss)
        
    def save(self):
        save_object(self.path, self.file_name, self.loss)
    
    def load(self):
        self.loss = load_object(self.path, self.file_name)

In [26]:
def train_one_epoch(model, train_loader, optim):

    running_loss = {
        "running_total_loss": 0,
        "running_loss_classifier": 0,
        "running_loss_box_reg": 0,
        "running_loss_objectness": 0,
        "running_loss_rpn_box_reg": 0
    }

    for images, targets in tqdm(train_loader, disable=DISABLE_TQDM):

        images_device = list(image.to(device) for image in images)
        targets_device = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optim.zero_grad()
        with torch.set_grad_enabled(True):
            loss_dict = model(images_device, targets_device)
            loss = sum(loss for loss in loss_dict.values())
            loss.backward()
            optim.step()

        running_loss["running_total_loss"] += loss.item() / len(train_loader)
        running_loss["running_loss_classifier"] += loss_dict['loss_classifier'].item() / len(train_loader)
        running_loss["running_loss_box_reg"] += loss_dict['loss_box_reg'].item() / len(train_loader)
        running_loss["running_loss_objectness"] += loss_dict['loss_objectness'].item() / len(train_loader)
        running_loss["running_loss_rpn_box_reg"] += loss_dict['loss_rpn_box_reg'].item() / len(train_loader)

    #epoch_total_loss = running_loss["running_total_loss"] / len(train_loader)
    #epoch_loss_classifier = running_loss["running_loss_classifier"] / len(train_loader)
    #epoch_loss_box_reg = running_loss["running_loss_box_reg"] / len(train_loader)
    #epoch_loss_objectness = running_loss["running_loss_objectness"] / len(train_loader)
    #epoch_loss_rpn_box_reg = running_loss["running_loss_rpn_box_reg"] / len(train_loader)

    print(running_loss)
    return running_loss

def evaluate_loss(model, loader):

    running_loss = {
        "running_total_loss": 0,
        "running_loss_classifier": 0,
        "running_loss_box_reg": 0,
        "running_loss_objectness": 0,
        "running_loss_rpn_box_reg": 0
    }

    for images, targets in tqdm(loader, disable=DISABLE_TQDM):
        images_device = list(image.to(device) for image in images)
        targets_device = [{k: v.to(device) for k, v in t.items()} for t in targets]
        model.train()
        with torch.set_grad_enabled(False):
            loss_dict = model.forward(images_device,targets_device)
            loss = sum(loss for loss in loss_dict.values())

        running_loss["running_total_loss"] += loss.item() / len(train_loader)
        running_loss["running_loss_classifier"] += loss_dict['loss_classifier'].item() / len(train_loader)
        running_loss["running_loss_box_reg"] += loss_dict['loss_box_reg'].item() / len(train_loader)
        running_loss["running_loss_objectness"] += loss_dict['loss_objectness'].item() / len(train_loader)
        running_loss["running_loss_rpn_box_reg"] += loss_dict['loss_rpn_box_reg'].item() / len(train_loader)

    print(running_loss)
    return running_loss

def evaluate_average_precision(model,loader):
    metric = MeanAveragePrecision(class_metrics=True)
    for images, targets in tqdm(loader, disable=DISABLE_TQDM):
        images_device = list(image.to(device) for image in images)
        targets_device = [{k: v.to(device) for k, v in t.items()} for t in targets]
        model.eval()
        with torch.set_grad_enabled(False):
            preds = model.forward(images_device)
            metric.update(preds, targets_device)
    mAP =  metric.compute()
    print(mAP)
    return mAP

In [27]:
def train_model(model, best_model, train_loader,valid_loader, optim, path=".",evaluate_map_every=5):

    model.train()

    train_history = LossHistory(path, "train_history")
    valid_history = LossHistory(path, "valid_history")
    mAP_history = []
    
    for i in range(1, n_epochs+1):
        print(f"\nEpoch {i}:")
        print("-"*100)

        ## Training
        print("Train")
        total_loss,classifier_loss,box_reg_loss,objectness_loss, rpn_box_reg_loss = train_one_epoch(model,train_loader,optim)
        train_history.update(total_loss,classifier_loss,box_reg_loss,objectness_loss, rpn_box_reg_loss)
        
        ## Validation
        print("\nValidation")
        total_loss,classifier_loss,box_reg_loss,objectness_loss, rpn_box_reg_loss = evaluate_loss(model,valid_loader)
        valid_history.update(total_loss,classifier_loss,box_reg_loss,objectness_loss, rpn_box_reg_loss)
        best_model.update(model,total_loss)
        
        ## Validation Eval mAP
        print("\nmAP Validation")
        if i%evaluate_map_every ==0:
            mAP = evaluate_average_precision(model,valid_loader)
            mAP_history.append(mAP)
            
        train_history.save(),
        valid_history.save()
        save_object(path,"map_history", mAP_history)
        
    return best_model.fetch(),train_history,valid_history,mAP_history

In [28]:
path = '/data/avramidi/tiny_vindr/'
train_path = path + "train_images/"
test_path  = path + "test_images/"
annot_path = path + "annotations/"

anomaly_map = {
    'No finding': 0,
    'Disc space narrowing': 1,
    'Foraminal stenosis': 2,
    'Osteophytes': 3,
    'Spondylolysthesis': 4,
    'Surgical implant': 5,
    'Vertebral collapse': 6,   
    'Other lesions': 7,
 }

In [16]:
train, valid, test = train_test_split(path)

train_annotations = pd.read_csv(annot_path + "train.csv")
test_annotations = pd.read_csv(annot_path + "test.csv")

train_annotation_map = compute_annotation_map(train_annotations)
train_annotation_map, valid_annotation_map = split_annotation_map(train_annotation_map, train, valid)

test_annotation_map = compute_annotation_map(test_annotations)

In [18]:
all_image_ids = [image.split('/')[-1].split('.')[0] for image in train] + \
                [image.split('/')[-1].split('.')[0] for image in valid] + \
                [image.split('/')[-1].split('.')[0] for image in test]

image_id_map = {
    img_id: i+1 for i, img_id in enumerate(all_image_ids)
}

In [19]:
train_transform = Compose([
    RandomHorizontalFlip(p=0.5),
    ToTensor()
])
test_transform = Compose([
    ToTensor()
])

train_dataset = SpineObjectDetection(train, train_annotation_map, anomaly_map, image_id_map, train_transform)
valid_dataset = SpineObjectDetection(valid, valid_annotation_map, anomaly_map, image_id_map, train_transform)
test_dataset  = SpineObjectDetection(test , test_annotation_map , anomaly_map, image_id_map, test_transform)

In [20]:
batch_size = 16

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=8)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=8)
test_loader  = data.DataLoader(test_dataset , batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=8)

In [23]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = create_faster_rcnn_model(num_classes=8, trainable_backbone_layers=3).to(device)
optim = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
n_epochs = 1
best_model = SaveBestModel("model_object.pt",path)

In [29]:
model, train_history, valid_history, mAP_history = train_model(
    model, best_model, train_loader, valid_loader, optim, path=path, evaluate_map_every=1
)


Epoch 1:
----------------------------------------------------------------------------------------------------
Train


RuntimeError: CUDA out of memory. Tried to allocate 364.00 MiB (GPU 1; 10.76 GiB total capacity; 9.01 GiB already allocated; 31.44 MiB free; 9.57 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [37]:
model = torch.load(path + "model_object.pt")
results = evaluate_average_precision(model,test_loader)
results